In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report

In [124]:
train_df = pd.read_csv("raw/train.csv")
test_df = pd.read_csv("raw/test.csv")

train_labels = pd.read_csv("raw/train_class_labels.csv").drop("Unnamed: 0", axis = 1)

In [123]:
# dropping columns
train_labels

,Unnamed: 0,income_>50K
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
43952,43952,1
43953,43953,0
43954,43954,0
43955,43955,0


In [139]:
"""Classes for transformations"""

class RemoveOutliers(BaseEstimator, TransformerMixin):
    """Removes outliers using IQR method"""

    def __init__(self, columns = []):
        self.columns = columns

    def fit(self, df, y = None):
        return self

    def transform(self, df, y = None):
        for col_no, col in enumerate(self.columns):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iq_range = q3 - q1
            df[col] = df[col].clip(lower = q1 - 1.5*iq_range, upper = q3 + 1.5*iq_range)
        return df

class AddPolyFeatures(BaseEstimator, TransformerMixin):
    """Add Polynomial features"""

    def __init__(self, columns = []):
        self.columns = columns
        self.rename_dict = dict()

    def fit(self, df, y = None):
        for i in range(df.shape[1]):
            self.rename_dict[i] = "pf_" + str(i)
        return self

    def transform(self, df, y = None):
        polynomial_features = PolynomialFeatures()
        polynomial_df = polynomial_features.fit_transform(df[self.columns])
        polynomial_df = polynomial_df.rename(self.rename_dict)
        return pd.merge(left = df, right = polynomial_df, left_index=True, right_index = True)

class MinMaxScale(BaseEstimator, TransformerMixin):
    """Normalizing Numerical Variables"""

    def __init__(self,columns=[]):
        self.columns = columns

    def fit(self, df, y = None):
        return self

    def transform(self, df, y = None):
        scaler = MinMaxScaler()
        df[self.columns] = scaler.fit_transform(df[self.columns])
        return df

class DropColumns(BaseEstimator, TransformerMixin):
    """For dropping unnecessary columns"""

    def __init__(self,columns=[]):
        self.columns = columns

    def fit(self, df, y = None):
        return self

    def transform(self, df, y = None):
        return df.drop(self.columns, axis = 1)

class OneHotEncode(BaseEstimator, TransformerMixin):
    """For One Hot Encoding categorical variables"""

    def __init__(self,columns=[]):
        self.columns = columns

    def fit(self, df, y = None):
        return self

    def transform(self,df, y = None):
        for i in self.columns:
            df[i+'i'] = df[i]
        return pd.get_dummies(df, columns = self.columns, drop_first=True)

class LabelEncode(BaseEstimator, TransformerMixin):
    """For Label Encoding Categorical Variables"""

    def __init__(self, columns = []):
        self.columns = columns
        self.encoders = []

    def fit(self, df, y = None):
        for i in self.columns:
            self.encoders.append(LabelEncoder())
        for i,col in enumerate(self.columns):
            self.encoders[i].fit(df[col])
        return self

    def transform(self, df, y = None):
        for col_no, i in enumerate(self.columns):
            df[i] = self.encoders[col_no].transform(df[i])
        return df

class Estimate(BaseEstimator, TransformerMixin):

    def __init__(self, estimator = "lg"):
        self.estimator = estimator

    def fit(self, y = None):
        return self

    def predict(self, df, y = None):
        if self.estimator == "lg":
            lr = LogisticRegression()
            lr.fit(df, y)
            return lr
        elif self.estimator == "svc":
            svc = SVC()
            svc.fit(df, y)
            return svc
        elif self.estimator == "dtc":
            dtc = DecisionTreeClassifier()
            dtc.fit(df, y)
            return dtc
        elif self.estimator == "rf":
            rf = RandomForestClassifier()
            rf.fit(df, y)
            return rf
        elif self.estimator == "knn":
            knn = KNeighborsClassifier()
            knn.fit(df, y)
            return knn
        elif self.estimator == "adt":
            adt = AdaBoostClassifier()
            adt.fit(df, y)
            return adt
        else:
            return None

In [140]:
pipeLine = Pipeline([("dropper", DropColumns(columns=['Unnamed: 0', 'education', 'capital-gain', 'capital-loss'])),
                     ("label_encoder", LabelEncode(columns=['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country'])),
                     ("minmaxscaler", MinMaxScale(columns=["age", "fnlwgt", "hours-per-week"])),
                     ("estimator", LogisticRegression())])

In [141]:
pipeLine.fit(X = train_df, y = train_labels)

C:\ProgramData\Anaconda3\envs\dsml\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\dsml\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('dropper',
                 DropColumns(columns=['Unnamed: 0', 'education', 'capital-gain',
                                      'capital-loss'])),
                ('label_encoder',
                 LabelEncode(columns=['workclass', 'marital-status',
                                      'occupation', 'relationship', 'race',
                                      'gender', 'native-country'])),
                ('minmaxscaler',
                 MinMaxScale(columns=['age', 'fnlwgt', 'hours-per-week'])),
                ('estimator', LogisticRegression())])

In [144]:
print(accuracy_score(pipeLine.predict(train_df), train_labels))

0.8075619355279022


In [109]:
pipeLine.transform(train_df)

,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country
0,0.684932,3,0.238967,16,0,3,1,4,1,0.602041,38
1,0.000000,3,0.156482,8,4,7,3,4,1,0.142857,38
2,0.191781,3,0.108814,13,2,3,0,4,1,0.397959,38
3,0.561644,6,0.065479,4,2,13,0,4,1,0.397959,38
4,0.109589,6,0.091919,10,4,7,1,2,1,0.397959,38
...,...,...,...,...,...,...,...,...,...,...,...
43952,0.479452,3,0.037572,13,2,3,0,4,1,0.500000,38
43953,0.027397,3,0.069788,9,4,7,3,4,0,0.397959,38
43954,0.178082,3,0.124893,10,0,11,1,4,1,0.581633,38
43955,0.397260,3,0.057140,13,4,11,1,4,0,0.346939,38
